In [ ]:
# Installation de XGBoost (souvent déjà installé dans Colab, mais c'est une bonne pratique)
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report

print("Bibliothèques importées avec succès.")

Bibliothèques importées avec succès.


Étape 1 : Chargement des Données et Inspection Rapide

In [ ]:
try:
    train_df = pd.read_csv("ressources/train.csv")
    test_df = pd.read_csv("ressources/test.csv")

    # Affichage des premières lignes pour vérifier le chargement
    print("--- TRAIN SET ---")
    print(train_df.head())
    print("\n--- TEST SET ---")
    print(test_df.head())

    # Vérification du déséquilibre (l'indice crucial de la détection de fraude)
    fraud_rate = train_df['is_fraud'].mean() * 100
    print(f"\nTaux de fraude dans le train set : {fraud_rate:.4f}% (Déséquilibre SÉVÈRE!)")

except FileNotFoundError:
    print("ERREUR : Fichiers 'train.csv' ou 'test.csv' introuvables. Vérifiez le nom et le chemin d'accès.")

--- TRAIN SET ---
                         transaction_id  step      type  amount customer_id  \
0  38d8cd1a-ab64-48a8-982b-547db096f8a5   421   PAYMENT  311000   C13646530   
1  2440928e-8a4c-4e74-a9c5-6329668be13d   325   PAYMENT  399600    C9452763   
2  973c2a3c-5676-477c-ac30-81334b01e90d   565   PAYMENT    7500   C31260354   
3  5413d0f4-b4f0-406f-bd91-91de1dce573c   511  TRANSFER   94000    C8841991   
4  c9fa99e2-42c8-4dd8-94ba-aa3e44bfdd98    63  CASH_OUT   15000    C3176716   

   age  is_fraud  
0   24         0  
1   25         0  
2   38         0  
3   26         0  
4   26         0  

--- TEST SET ---
                         transaction_id  step      type   amount customer_id  \
0  f9d47afa-6e9f-45a1-a443-afb5132e9986   494   PAYMENT   184500   C57389453   
1  a3159433-df38-44d2-b6e1-3920b6f2a42f   322   PAYMENT   684000   C15998978   
2  27b30f3c-7688-489f-9c77-d70981bf1a05   377  CASH_OUT  1344000   C48225516   
3  c3566535-87b7-40d5-ac21-7874a193af11   592   PAYMENT

Étape 2 : Feature Engineering

In [ ]:
def create_temporal_features(df):
    """Création des features temporelles (jour, heure, nuit, weekend) à partir de 'step'."""

    # Le Step 1 est la première heure d'un LUNDI (jour 0)
    df['day_of_week'] = ((df['step'] - 1) // 24) % 7 # 0=Lundi, 6=Dimanche
    df['hour'] = (df['step'] - 1) % 24 # 0 à 23

    # 1 si la transaction a lieu la nuit (ex: 0h à 6h)
    df['is_night_time'] = ((df['hour'] >= 0) & (df['hour'] <= 6)).astype(int)
    # 1 si la transaction a lieu le week-end (Samedi ou Dimanche)
    df['is_weekend'] = ((df['day_of_week'] >= 5)).astype(int)

    return df

def advanced_feature_engineering(df):
    """Création des features basées sur le comportement du client (agrégations)."""

    # Fréquence des transactions : Combien de fois le client a transigé dans cette fenêtre de 24h
    df['step_24h_window'] = df['step'] // 24
    df['client_txn_count_24h'] = df.groupby(['customer_id', 'step_24h_window'])['amount'].transform('count')

    # Ratio d'anomalie : Montant de la transaction actuelle VS Montant moyen habituel pour ce type de transaction par ce client
    mean_amount_by_client_type = df.groupby(['customer_id', 'type'])['amount'].transform('mean')
    df['amount_vs_mean_type'] = df['amount'] / (mean_amount_by_client_type + 1e-6) # Ajout d'epsilon pour éviter la division par zéro

    # Indicateur pour les types à haut risque
    df['is_transfer_or_cashout'] = df['type'].apply(lambda x: 1 if x in ['TRANSFER', 'CASH_OUT'] else 0)

    # Suppression de la colonne temporaire
    df = df.drop('step_24h_window', axis=1)

    return df

# Application des fonctions à l'entraînement et au test
train_df = create_temporal_features(train_df.copy())
train_df = advanced_feature_engineering(train_df)

test_df_submission = test_df.copy() # Sauvegarde pour la soumission
test_df = create_temporal_features(test_df.copy())
test_df = advanced_feature_engineering(test_df)

# Séparation de la cible (y) et des features (X)
X = train_df.drop(['is_fraud', 'transaction_id', 'customer_id', 'step'], axis=1)
y = train_df['is_fraud']
X_test_final = test_df.drop(['transaction_id', 'customer_id', 'step'], axis=1)

# Division pour la validation (on garde 20% du train pour tester le modèle)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("✅ Feature Engineering terminé et données séparées.")

✅ Feature Engineering terminé et données séparées.


Étape 3 : Préprocesseur et Baseline (Régression Logistique)

In [ ]:
# --- Définition des Features pour le Préprocesseur ---
numerical_features = ['amount', 'age', 'client_txn_count_24h', 'amount_vs_mean_type']
categorical_features = ['type', 'day_of_week', 'hour']
boolean_features = ['is_night_time', 'is_weekend', 'is_transfer_or_cashout']

# ColumnTransformer : Applique les bonnes transformations aux bonnes colonnes
preprocessor = ColumnTransformer(
    transformers=[
        # StandardScaler : Met les colonnes numériques à la même échelle (important pour la Rég. Log.)
        ('num', StandardScaler(), numerical_features),
        # OneHotEncoder : Convertit les variables catégorielles (ex: 'type') en colonnes numériques (0 ou 1)
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('bool', 'passthrough', boolean_features) # Laisse les 0/1 tranquilles
    ],
    remainder='drop'
)

# --- Pipeline de Baseline (Régression Logistique) ---
log_reg_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # LogisticRegression avec class_weight='balanced' pour gérer le déséquilibre
    ('classifier', LogisticRegression(random_state=42, class_weight='balanced', solver='liblinear', max_iter=1000))
])

# Entraînement et Évaluation de la Baseline
print("🔄 Entraînement de la Baseline (Rég. Logistique)...")
log_reg_model.fit(X_train, y_train)

y_pred_baseline = log_reg_model.predict(X_val)

print("\n--- Résultats de la Baseline ---")
f1_baseline = f1_score(y_val, y_pred_baseline)
print(f"F1-Score Baseline: **{f1_baseline:.4f}**")
print("\nMatrice de Confusion (Baseline) :\n", confusion_matrix(y_val, y_pred_baseline))
print("\nRapport de Classification (Baseline) :\n", classification_report(y_val, y_pred_baseline))

🔄 Entraînement de la Baseline (Rég. Logistique)...

--- Résultats de la Baseline ---
F1-Score Baseline: **0.1416**

Matrice de Confusion (Baseline) :
 [[4569 1312]
 [  10  109]]

Rapport de Classification (Baseline) :
               precision    recall  f1-score   support

           0       1.00      0.78      0.87      5881
           1       0.08      0.92      0.14       119

    accuracy                           0.78      6000
   macro avg       0.54      0.85      0.51      6000
weighted avg       0.98      0.78      0.86      6000



Étape 4 : Modèle Avancé (XGBoost) et Optimisation

In [ ]:
# --- Calcul du poids pour la classe positive (Gestion du Déséquilibre pour XGBoost) ---
# On dit à XGBoost que la classe '1' (fraude) est beaucoup plus importante
scale_pos_weight_value = sum(y_train == 0) / sum(y_train == 1)

# --- Pipeline XGBoost ---
xgb_classifier = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    n_estimators=200, # Un peu plus d'arbres
    learning_rate=0.08, # Un taux d'apprentissage modéré
    scale_pos_weight=scale_pos_weight_value, # Appliquer le poids calculé
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False
)

xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb_classifier)
])

# Entraînement du Modèle Final
print("\n🔄 Entraînement du Modèle Final (XGBoost)...")
xgb_model.fit(X_train, y_train)

# Évaluation du Modèle Final
y_pred_xgb = xgb_model.predict(X_val)

print("\n--- Résultats du Modèle Final (XGBoost) ---")
f1_xgb = f1_score(y_val, y_pred_xgb)
print(f"F1-Score Final (XGBoost): **{f1_xgb:.4f}**")
print("\nMatrice de Confusion (XGBoost) :\n", confusion_matrix(y_val, y_pred_xgb))
print("\nRapport de Classification (XGBoost) :\n", classification_report(y_val, y_pred_xgb))


🔄 Entraînement du Modèle Final (XGBoost)...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [06:21:59] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



--- Résultats du Modèle Final (XGBoost) ---
F1-Score Final (XGBoost): **0.7117**

Matrice de Confusion (XGBoost) :
 [[5819   62]
 [  19  100]]

Rapport de Classification (XGBoost) :
               precision    recall  f1-score   support

           0       1.00      0.99      0.99      5881
           1       0.62      0.84      0.71       119

    accuracy                           0.99      6000
   macro avg       0.81      0.91      0.85      6000
weighted avg       0.99      0.99      0.99      6000



Étape 5 : Génération de la Soumission Finale

In [ ]:
# 1. Entraînement final sur l'ensemble des données d'entraînement (X complet, y complet)
print("\n🔄 Entraînement final du XGBoost sur l'intégralité du TRAIN SET...")
xgb_model.fit(X, y)

# 2. Prédictions sur le TEST SET
print("🚀 Génération des prédictions pour la soumission...")
test_predictions = xgb_model.predict(X_test_final)

# 3. Création du DataFrame de soumission
submission = pd.DataFrame({
    # On utilise les IDs originaux du test set
    "transaction_id": test_df_submission["transaction_id"],
    # La prédiction est convertie en entier (0 ou 1)
    "is_fraud": test_predictions.astype(int)
})

# 4. Sauvegarde du fichier submission.csv
submission.to_csv("submission.csv", index=False)

print(f"\n✅ Fichier submission.csv généré avec {len(submission)} prédictions. Vous pouvez le télécharger dans l'onglet 'Fichiers' de Colab.")


🔄 Entraînement final du XGBoost sur l'intégralité du TRAIN SET...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [06:22:55] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🚀 Génération des prédictions pour la soumission...

✅ Fichier submission.csv généré avec 10000 prédictions. Vous pouvez le télécharger dans l'onglet 'Fichiers' de Colab.
